In [1]:
import requests
from bs4 import BeautifulSoup
import gspread
from datetime import datetime

In [6]:
### Scrape
## Config Details
app_url = 'https://finance.yahoo.com/quote/APP/holders/'
agent = 'Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'


## Scrape
response = requests.get(app_url, headers={'User-Agent': agent})
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
soup

<html><meta charset="utf-8"/><script>if(window!=window.top){document.write('<p>Content is currently unavailable.</p><img src="//geo.yahoo.com/p?s=1197757039&t='+new Date().getTime()+'&_R='+encodeURIComponent(document.referrer)+'&err=404&err_url='+'https%3A%2F%2Fbrb.yahoo.net%3A443%2Ffinance.yahoo.com%2Fdesktop%2Fquote%2FAPP%2Fholders%2Findex.html'+'" width="0px" height="0px"/>');}else{window.location.replace('https://www.yahoo.com/?err=404&err_url=https%3A%2F%2Fbrb.yahoo.net%3A443%2Ffinance.yahoo.com%2Fdesktop%2Fquote%2FAPP%2Fholders%2Findex.html');}</script><noscript><meta content="0;URL='https://www.yahoo.com/?err=404&amp;err_url=https%3A%2F%2Fbrb.yahoo.net%3A443%2Ffinance.yahoo.com%2Fdesktop%2Fquote%2FAPP%2Fholders%2Findex.html'" http-equiv="refresh"/></noscript></html>

In [4]:
## Dissect
# Locate the section containing the "Major Holders" table by the section's `data-testid` attribute
section = soup.find('section', {'data-testid': 'holders-major-holders-table'})
# Find all 'td' elements within the section to extract the data
td_elements = section.find_all('td', class_='majorHolders')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [9]:
import pandas as pd

short_url = "https://finance.yahoo.com/quote/APP/holders/"
pd.read_html(short_url)

HTTPError: HTTP Error 404: Not Found

In [ ]:
## Load dict
# Initialize an empty dictionary
holders_dict = {}

# Iterate through the list in pairs (numeric value and its corresponding label)
for i in range(0, len(td_elements), 2):
    # Strip any extra characters and clean up the value
    value = td_elements[i].text.strip().replace('%', '')
    
    # Convert the value to float or integer as appropriate
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    
    # Get the corresponding key (label)
    label = td_elements[i+1].text.strip()
    
    # Update the dictionary
    holders_dict[label] = value

In [ ]:
## Stock Price
price_element = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'})

# Extract the value from the 'data-value' attribute
stock_price = price_element['data-value']

In [ ]:
### Send to GSheet
## Config details
sa_path = r"C:\Users\Owner\AppData\Local\Programs\Python\Python310\Lib\site-packages\gspread\service_account.json"
gc = gspread.service_account(filename=sa_path)


## Open the sheet
sh = gc.open("APP Ownership")
worksheet = sh.worksheet("Data")  # Access the "Data" sheet

In [ ]:
## Insert data
# Extract the values in the correct order (A, B, C, D, E, F)
values = [
    stock_price,
    holders_dict["% of Shares Held by All Insider"],
    holders_dict["% of Shares Held by Institutions"],
    holders_dict["% of Float Held by Institutions"],
    holders_dict["Number of Institutions Holding Shares"],
    datetime.now().strftime("%Y/%m/%d %H:%M:%S") + " MT"
]

# Insert a new row at position 2 (shifts existing row 2 and below down)
worksheet.insert_row([], 2)

# Update row 2 with the new data in columns A to F
worksheet.update('A2:F2', [values])

# Add formulas individually to columns F:I for the new row using update_acell (all together came through as a string in the gsheet)
worksheet.update_acell('G2', '=IFERROR(((A2-A3)/A3),"")')
worksheet.update_acell('H2', '=IFERROR(((B2-B3)/B3),"")')
worksheet.update_acell('I2', '=IFERROR(((C2-C3)/C3),"")')
worksheet.update_acell('J2', '=IFERROR(((D2-D3)/D3),"")')
worksheet.update_acell('K2', '=IFERROR(((E2-E3)/E3),"")')

## Try selenium

In [1]:
import pandas as pd

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

from datetime import datetime

import gspread

In [2]:
# Get the ChromeDriver path from your environment variable
chrome_driver_path = os.getenv('chrome_driver_path')
app_url = "https://finance.yahoo.com/quote/APP/holders/"

In [3]:
# Set up Chrome options for headless mode
options = Options()
options.add_argument('--headless') # Run in headless mode ('--headless=new')
options.add_argument("--window-size=1920,1080")  # Optional: set the window size for better layout handling
options.add_argument("--no-sandbox")  # Required for some environments
options.add_argument("--disable-dev-shm-usage")  # Optional: improve stability in headless mode

In [4]:
# Setup WebDriver
service = Service(chrome_driver_path)  # Use the path from environment variable
driver = webdriver.Chrome(service=service, options=options)

# Open the page
driver.get(app_url)

# Now proceed with your scraping task
## Use BeautifulSoup to parse the page source once the page is fully loaded
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close out
driver.quit()

In [5]:
## Dissect
# Locate the section containing the "Major Holders" table by the section's `data-testid` attribute
section = soup.find('section', {'data-testid': 'holders-major-holders-table'})
# Find all 'td' elements within the section to extract the data
td_elements = section.find_all('td', class_='majorHolders')

In [6]:
## Load dict
# Initialize an empty dictionary
holders_dict = {}

# Iterate through the list in pairs (numeric value and its corresponding label)
for i in range(0, len(td_elements), 2):
    # Strip any extra characters and clean up the value
    value = td_elements[i].text.strip().replace('%', '')
    
    # Convert the value to float or integer as appropriate
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    
    # Get the corresponding key (label)
    label = td_elements[i+1].text.strip()
    
    # Update the dictionary
    holders_dict[label] = value

## Stock Price
# Look for the `fin-streamer` tag with both `data-field="regularMarketPrice"` and `data-symbol="APP"`
price_element = soup.find('fin-streamer', {'data-field': 'regularMarketPrice', 'data-symbol': 'APP'})

# Extract the value from the 'data-value' attribute
stock_price = price_element['data-value']

In [7]:
# Extract the values in the correct order (A, B, C, D, E, F)
values = [
    stock_price,
    holders_dict["% of Shares Held by All Insider"],
    holders_dict["% of Shares Held by Institutions"],
    holders_dict["% of Float Held by Institutions"],
    holders_dict["Number of Institutions Holding Shares"],
    datetime.now().strftime("%Y/%m/%d %H:%M:%S") + " MT"
]

In [8]:
values

['160.005', 27.69, 59.19, 81.86, 898, '2024/11/04 09:58:08 MT']

In [ ]:
### Send to GSheet
## Config details
sa_path = r"C:\Users\Owner\AppData\Local\Programs\Python\Python310\Lib\site-packages\gspread\service_account.json"
gc = gspread.service_account(filename=sa_path)


## Open the sheet
sh = gc.open("APP Ownership")
worksheet = sh.worksheet("Data")  # Access the "Data" sheet


# Insert a new row at position 2 (shifts existing row 2 and below down)
worksheet.insert_row([], 2)

# Update row 2 with the new data in columns A to F
worksheet.update([values], 'A2:F2')

# Add formulas individually to columns F:I for the new row using update_acell (all together came through as a string in the gsheet)
worksheet.update_acell('G2', '=IFERROR(((A2-A3)/A3),"")')
worksheet.update_acell('H2', '=IFERROR(((B2-B3)/B3),"")')
worksheet.update_acell('I2', '=IFERROR(((C2-C3)/C3),"")')
worksheet.update_acell('J2', '=IFERROR(((D2-D3)/D3),"")')
worksheet.update_acell('K2', '=IFERROR(((E2-E3)/E3),"")')

{'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
 'updatedRange': 'Data!K2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

# Selenium on Mac

## Try selenium

In [1]:
import pandas as pd

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

from datetime import datetime

import gspread

In [2]:
# Get the ChromeDriver path from your environment variable
chrome_driver_path = os.getenv('chrome_driver_path')
app_url = "https://finance.yahoo.com/quote/APP/holders/"

In [3]:
# Set up Chrome options for headless mode
options = Options()
options.add_argument('--headless') # Run in headless mode ('--headless=new')
options.add_argument("--window-size=1920,1080")  # Optional: set the window size for better layout handling
options.add_argument("--no-sandbox")  # Required for some environments
options.add_argument("--disable-dev-shm-usage")  # Optional: improve stability in headless mode

In [4]:
# Setup WebDriver
service = Service(chrome_driver_path)  # Use the path from environment variable
driver = webdriver.Chrome(service=service, options=options)

# Open the page
driver.get(app_url)

# Now proceed with your scraping task
## Use BeautifulSoup to parse the page source once the page is fully loaded
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close out
driver.quit()

In [5]:
## Dissect
# Locate the section containing the "Major Holders" table by the section's `data-testid` attribute
section = soup.find('section', {'data-testid': 'holders-major-holders-table'})
# Find all 'td' elements within the section to extract the data
td_elements = section.find_all('td', class_='majorHolders')

In [6]:
## Load dict
# Initialize an empty dictionary
holders_dict = {}

# Iterate through the list in pairs (numeric value and its corresponding label)
for i in range(0, len(td_elements), 2):
    # Strip any extra characters and clean up the value
    value = td_elements[i].text.strip().replace('%', '')
    
    # Convert the value to float or integer as appropriate
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    
    # Get the corresponding key (label)
    label = td_elements[i+1].text.strip()
    
    # Update the dictionary
    holders_dict[label] = value

## Stock Price
# Look for the `fin-streamer` tag with both `data-field="regularMarketPrice"` and `data-symbol="APP"`
price_element = soup.find('fin-streamer', {'data-field': 'regularMarketPrice', 'data-symbol': 'APP'})

# Extract the value from the 'data-value' attribute
stock_price = price_element['data-value']

In [7]:
# Extract the values in the correct order (A, B, C, D, E, F)
values = [
    stock_price,
    holders_dict["% of Shares Held by All Insider"],
    holders_dict["% of Shares Held by Institutions"],
    holders_dict["% of Float Held by Institutions"],
    holders_dict["Number of Institutions Holding Shares"],
    datetime.now().strftime("%Y/%m/%d %H:%M:%S") + " MT"
]

In [8]:
values

['168.375', 27.73, 61.17, 84.65, 910, '2024/11/06 10:31:03 MT']

In [9]:
### Send to GSheet
## Config details
sa_path_windows = r"C:\Users\Owner\AppData\Local\Programs\Python\Python310\Lib\site-packages\gspread\service_account.json"
sa_path_mac = "/Users/ckopel/Documents/keys/service_account.json"

sa_path = sa_path_windows if os.name == "nt" else sa_path_mac if os.name == "posix" else None

gc = gspread.service_account(filename=sa_path)


## Open the sheet
sh = gc.open("APP Ownership")
worksheet = sh.worksheet("Data")  # Access the "Data" sheet


# Insert a new row at position 2 (shifts existing row 2 and below down)
worksheet.insert_row([], 2)

# Update row 2 with the new data in columns A to F
worksheet.update([values], 'A2:F2')

# Add formulas individually to columns F:I for the new row using update_acell (all together came through as a string in the gsheet)
worksheet.update_acell('G2', '=IFERROR(((A2-A3)/A3),"")')
worksheet.update_acell('H2', '=IFERROR(((B2-B3)/B3),"")')
worksheet.update_acell('I2', '=IFERROR(((C2-C3)/C3),"")')
worksheet.update_acell('J2', '=IFERROR(((D2-D3)/D3),"")')
worksheet.update_acell('K2', '=IFERROR(((E2-E3)/E3),"")')

{'spreadsheetId': '1WLCqIuF9P7_ypl_hF-9QtwaFr2-jjl0DDNgpB9HfB7A',
 'updatedRange': 'Data!K2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}